<a href="https://colab.research.google.com/github/memoandrea/Sinandah/blob/main/deep_learning_with_large_files_incremental_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#INCREMENTAL LEARNING AND RANDOM SAMPLING

"""generate some data, build a subsample function  to draw random subsamples  from the dataset and
incrementally train a deep learning model on these subsets"""

#import neccessary modules

!pip uninstall skflow -y
!pip install tensorflow
import numpy as np
import pandas as pd

from sklearn.datasets import make_classification
import random
from sklearn.model_selection import train_test_split
import gc
import tensorflow as tf
from sklearn.metrics import accuracy_score

In [16]:
#GENERATE SOME EXAMPLE DATA AND WRITE IT TO DISK

x, y = make_classification(n_samples = 5000000, n_features = 10,
                           n_classes = 4, n_informative = 6, random_state = 123, n_clusters_per_class = 1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 555)

Big_trainM = pd.DataFrame(x_train, y_train)
Big_testM = pd.DataFrame(x_test, y_test)

# Use parentheses to call to_csv and '=' to assign separator
Big_trainM.to_csv('lsml-BigtrainM', sep=',')
Big_testM.to_csv('lsml-BigtestM', sep=',')

In [8]:
#FREE MEMORY BY FREEING THE OBJECTS CREATED

del(x, y, x_train, y_train)
#force python's garbage collector to empty memory
gc.collect

<function gc.collect(generation=2)>

In [14]:
#CREATE A FUNCTION THAT DRAWS RANDOM SUBSAMPLES FRO DISK

big_train = 'lsml-BigtrainM'

import pandas as pd
import random
def sample_file():
  global skip_idx
  global train_data
  global x_train
  global y_train



In [20]:
#COUNT THE NUMBER OF ROWS IN THE ENTIRE SET

num_lines = sum(1 for i in open(big_train))
print(num_lines)
#third fraction of the training set
size = int(num_lines/3)

4000001


In [34]:
#SKIP INDEXES AND KEEP INDICES

skip_idx = random.sample(range = (1, num_lines), num_lines - size)
train_data = pd.read_csv(big_train, skip_rows = skip_idx)
x_train = train_data.drop(train_data.columns[[0]], axis = 1)
y_train = train_data.ix[:,0]

SyntaxError: positional argument follows keyword argument (<ipython-input-34-e6393a3e92a7>, line 3)

In [28]:
#USE WEIGHT DECAY

def exp_decay(global_step):
  return tf.train.exponential_decay(
      learning_rate = 0.01, global_step = global_step, decay_steps = steps, decay_rate = 0.01
  )

In [38]:
#SET UP THE DEEP NEURAL NETWORK CLASSIFIER

""" 3 hidden layers: 5,4,4 units
batch size: 300 300training cases in each epoch to prevent memory overloading"""
!pip install tensorflow # install tensorflow if you haven't already
import tensorflow as tf
import tensorflow_estimator as tfe

steps = 5000

clf = tfe.DNNClassifier(
    hidden_units = [5, 4, 4],
    n_classes = 4,
    optimizer = tf.train.AdamOptimizer(learning_rate = exp_decay),
    batch_size = 300
)



AttributeError: module 'tensorflow_estimator' has no attribute 'DNNClassifier'

In [40]:
#SET THE AMOUNT OF SUBSAMPLES(EPOCHS)

"""incrementally train the DNN on three consecutive subsamples"""

epochs = 3
for i in range(epochs):
  sample_file()
  clf.partial_fit(x_train, y_train)

NameError: name 'clf' is not defined

In [42]:
#EVALUATION

test_data = pd.read_csv('lsml-BigtrainM', sep = ',')
x_test = test_data.drop(test_data.columns[[0]], axis = 1)
y_test = test_data.ix[:, 0]
score = accuracy_score(y_test, clf.predict(x_test))
print(score)

AttributeError: 'DataFrame' object has no attribute 'ix'